In [7]:
# pip install faiss-cpu
# pip install torch
# pip install tensorflow
# pip install transformers

# !pip install sentence_transformers
# !pip install pandas

In [7]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

In [8]:
df = pd.read_csv("../Crawling/서울시 공공데이터 최종.csv")
# df.head().T

service_name = df['서비스명'].to_list()[:100]
service_dscrp = df['서비스설명'].to_list()[:100]

In [24]:
print('총 샘플의 개수 :', len(service_name))
print('총 샘플의 개수 :', len(service_dscrp))

총 샘플의 개수 : 100
총 샘플의 개수 : 100


In [25]:
# 임베딩
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(service_name)
print('임베딩 된 벡터 수 :', len(encoded_data))

임베딩 된 벡터 수 : 100


In [26]:
# 인덱스 정의 및 데이터 추가
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(service_name))))

faiss.write_index(index, 'abc_news')

In [27]:
def search(query):
    t = time.time()
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    print('total time: {}'.format(time.time() - t))
    return [service_name[_id] for _id in top_k[1].tolist()[0]]

In [28]:
query = str(input())
results = search(query)

print('results :')
for result in results:
    print('\t', result)

In [2]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import tensorflow as tf

/Users/ktg/miniforge3/envs/fastapi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('klue/bert-base')
model = BartForConditionalGeneration.from_pretrained('klue/bert-base')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [3]:
text = 'You passed along `num_labels=3` with an incompatible id to label map'

In [4]:
raw_input_ids = tokenizer.encode(text)

In [3]:
# raw_input_ids

In [17]:
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np
import torch

class PseudoTextSearchSystem:
    def __init__(self):
        # self.tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
        # self.model = AutoModel.from_pretrained("kykim/bert-kor-base")
        self.tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
        self.model = AutoModel.from_pretrained("klue/bert-base")
        self.index = None
        self.text_db = []
        
    def add_text_to_db(self, text):
        self.text_db.append(text)
        
    def build_index(self):
        embeddings = self._get_embeddings(self.text_db)
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)
        
    def search_text(self, query, k=5):
        if self.index is None:
            raise ValueError("Index is not built yet. Call build_index() first.")
        
        query_embedding = self._get_embeddings([query])[0]
        distances, indices = self.index.search(np.array([query_embedding]), k)
        
        print('distances')
        print(distances)
        print('indices')
        print(indices)
        
        results = [(self.text_db[idx], score) for idx, score in zip(indices[0], distances[0])]
        return results
    
    def _get_embeddings(self, texts):
        encoded_input = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
            embeddings = model_output.last_hidden_state[:, 0, :].numpy()
        return embeddings

In [18]:
# Create the search system
search_system = PseudoTextSearchSystem()

# Add texts to the database
# search_system.add_text_to_db("안녕하세요, 반갑습니다.")
# search_system.add_text_to_db("오늘 날씨가 좋네요.")
# search_system.add_text_to_db("저는 파이썬 개발자입니다.")

# for txt in service_name:
#     search_system.add_text_to_db(txt)

# Build the search index
search_system.build_index()

# Search for similar texts
results = search_system.search_text("서대문구")
print(results)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


distances
[[501.05188 505.79608 520.9692  524.95544 532.3541 ]]
indices
[[97 23 34 41 93]]
[('서울시 서대문구 게임물제작업 인허가 정보', 501.05188), ('서울시 동대문구 체력단련장업 인허가 정보', 505.79608), ('서울시 서대문구 전체 행정처분내역 현황 정보', 520.9692), ('서울시 동대문구 노동조합 현황', 524.95544), ('서울시 용산구 어린이집 정보 (표준 데이터)', 532.3541)]


In [34]:
faiss.write_index()

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x17c0d0570> >

In [ ]:
filename = 'embedding_vectors.pkl'
with open(filename, 'wb') as f:
    embedding_vectors = pickle.load(f)